In [1]:
import subprocess
import datetime
import math
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import inspect

In [8]:
initial_cost_parameters = {"It_PV_1kW[yen/year]": [726383.3333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                           "Mt_PV_1kW[yen/year]":[4329] * 20,
                           "Ft_PV_1kW[yen/year]":[0]*20,
                           "It_Wind_1kW[yen/year]": [302274.7826,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                           "Mt_Wind_1kW[yen/year]":[662.785213] * 20,
                           "Ft_Wind_1kW[yen/year]":[0]*20,
                           "It_Diesel_1kW[yen/year]": [726383.3333,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
                           "Mt_Diesel_1kW[yen/year]":[8.8] * 20,
                           "It_Battery_1kW[yen/year]": [13540,0,0,0,13540,0,0,0,13540,0,0,0,13540,0,0,0,13540,0,0,0],
                           "Mt_Battery_1kW[yen/year]":[1100] * 20,
                           "Ft_Battery_1kW[yen/year]":[0]*20,
                           "r[yen/year]":[0.0234375]*20,
                           "operation_year":list(range(1,21)),
                           }

cost_parameters={
    "(1+r)^t":(1+np.array(initial_cost_parameters["r[yen/year]"] ) **initial_cost_parameters["operation_year"]).tolist(),
}

Et=[10535.179393434208]*20

In [9]:
variables = {'pv_cap_max': 81.36994985098434, 'wind_cap_max': 71.07446859887513, 'battery_cap_max': 89.6328267254965, 'battery_max': 80.66954405294685, 'battery_min': 17.926565345099302, 'diesel_max': 94.95165119915434, 'demand_sum': 38.687999999999995,
                 'pv_power_sum': 0.6428243995539474, 'wind_power_sum': 83.20371806928155, 'battery_charging_power_sum': 60.84095284768869, 'battery_discharging_power_sum': 26.204170875926295, 'diesel_power_sum': 0, 'trashed_power_sum': 10.521760497073117}

print(variables.keys())
print(type(variables["pv_cap_max"]))
print(initial_cost_parameters.keys())

dict_keys(['pv_cap_max', 'wind_cap_max', 'battery_cap_max', 'battery_max', 'battery_min', 'diesel_max', 'demand_sum', 'pv_power_sum', 'wind_power_sum', 'battery_charging_power_sum', 'battery_discharging_power_sum', 'diesel_power_sum', 'trashed_power_sum'])
<class 'float'>
dict_keys(['It_PV_1kW[yen/year]', 'Mt_PV_1kW[yen/year]', 'Ft_PV_1kW[yen/year]', 'It_Wind_1kW[yen/year]', 'Mt_Wind_1kW[yen/year]', 'Ft_Wind_1kW[yen/year]', 'It_Diesel_1kW[yen/year]', 'Mt_Diesel_1kW[yen/year]', 'It_Battery_1kW[yen/year]', 'Mt_Battery_1kW[yen/year]', 'Ft_Battery_1kW[yen/year]', 'r[yen/year]', 'operation_year'])


In [14]:
SCL = (((variables["pv_cap_max"]/1000)
     * (np.array(initial_cost_parameters["It_PV_1kW[yen/year]"])+np.array(initial_cost_parameters["Mt_PV_1kW[yen/year]"])))\
    + (variables["wind_cap_max"]\
    * (np.array(initial_cost_parameters["It_Wind_1kW[yen/year]"])+np.array(initial_cost_parameters["Mt_Wind_1kW[yen/year]"]))) \
    + (variables["diesel_max"]\
    * (np.array(initial_cost_parameters["It_Diesel_1kW[yen/year]"])+np.array(initial_cost_parameters["Mt_Diesel_1kW[yen/year]"]) ) ) \
    + (variables["battery_cap_max"]\
    * (np.array(initial_cost_parameters["It_Battery_1kW[yen/year]"])+np.array(initial_cost_parameters["Mt_Battery_1kW[yen/year]"]))) \
      +variables["battery_cap_max"]) \
    /cost_parameters["(1+r)^t"]

SEL =np.array(Et)/cost_parameters["(1+r)^t"]
print((variables["pv_cap_max"]
     * (np.array(initial_cost_parameters["It_PV_1kW[yen/year]"])+np.array(initial_cost_parameters["Mt_PV_1kW[yen/year]"]))))   
LCOE=np.sum(SCL)/np.sum(SEL)
LCOE

[59458025.91611676   352250.51290491   352250.51290491   352250.51290491
   352250.51290491   352250.51290491   352250.51290491   352250.51290491
   352250.51290491   352250.51290491   352250.51290491   352250.51290491
   352250.51290491   352250.51290491   352250.51290491   352250.51290491
   352250.51290491   352250.51290491   352250.51290491   352250.51290491]


462.88963628053193

In [11]:
def calc_cost(variables, initial_cost_parameters):
    Et = [variables["pv_power_sum"]+variables["wind_power_sum"] +
          variables["diesel_power_sum"]]*20
    cost_parameters = {
        "(1+r)^t": (1+np.array(initial_cost_parameters["r[yen/year]"]) ** initial_cost_parameters["operation_year"]).tolist(),
    }
    SCL = ((variables["pv_cap_max"]
            * (np.array(initial_cost_parameters["It_PV_1kW[yen/year]"])+np.array(initial_cost_parameters["Mt_PV_1kW[yen/year]"])))
           + (variables["wind_cap_max"]
              * (np.array(initial_cost_parameters["It_Wind_1kW[yen/year]"])+np.array(initial_cost_parameters["Mt_Wind_1kW[yen/year]"])))
           + (variables["diesel_max"]
              * (np.array(initial_cost_parameters["It_Diesel_1kW[yen/year]"])+np.array(initial_cost_parameters["Mt_Diesel_1kW[yen/year]"])
                 + np.array(initial_cost_parameters["Ft_Diesel_1kW[yen/kWh/year]"])))
           + (variables["battery_cap_max"]
              * (np.array(initial_cost_parameters["It_Battery_1kW[yen/year]"])+np.array(initial_cost_parameters["Mt_Battery_1kW[yen/year]"])))) \
        / cost_parameters["(1+r)^t"]
    
    SEL =np.array(Et)/cost_parameters["(1+r)^t"]
    LCOE=np.sum(SCL)/np.sum(SEL)
    return LCOE

In [6]:
def calc_cost(variables, initial_cost_parameters):
    #initial_cost_parameters["PV_cost[yen/kWh]"]
    #initial_cost_parameters["battery_cost[yen/kWh]"]
    #initial_cost_parameters["diesel_cost[yen/kWh]"]
    total_cost = variables["pv_power_sum"] * initial_cost_parameters["PV_cost[yen/kWh]"] \
                +variables["wind_power_sum"] * initial_cost_parameters["Wind_cost[yen/kWh]"] \
                +(variables["battery_charging_power_sum"] + variables["battery_discharging_power_sum"] ) \
                * initial_cost_parameters["battery_cost[yen/kWh]"] \
                +variables["diesel_power_sum"] * initial_cost_parameters["diesel_cost[yen/kWh]"]

    return total_cost

In [7]:
total_cost=calc_cost(variables, initial_cost_parameters)
total_cost

KeyError: 'PV_cost[yen/kWh]'